### **Grand Canyon Sandbar Timelapse creator**

A script to create timelapses from sandbar imagery (testing script)

Created by Ryan E. Lima ryan.lima@nau.edu for the USGS GCMRC

#### To Do List
1.  Turn into python Package
2.  improve commenting
3.  create in-depth readme
4.  add error messages
5.  provide sample dataset

In [35]:
import re
import os
import cv2
import glob
from datetime import datetime



In [36]:
##------------ get list of images----------------------------#
# sample image directories from sandbars for which timelapses will be made
image_dir30 ='M:/Ryan_Lima/Remote Cameras/full_res/RC0307Rf' # location on local PC or attached drive
image_dir22_web = 'M:/Ryan_Lima/Remote Cameras/web_res/RC0220Ra' # location on local PC or attached drive


def get_image_files(directory):
    """
    Get a list of image file paths from the given directory.
    Supports .jpg, .JPG, .jpeg, .JPEG file extensions.
    """
    extensions = ['jpg', 'JPG', 'jpeg', 'JPEG']
    files = []
    for ext in extensions:
        files.extend(glob.glob(os.path.join(directory, f'*.{ext}')))
    return files

image_files = get_image_files(image_dir30)
print(image_files[1])

M:/Ryan_Lima/Remote Cameras/full_res/RC0307Rf\RC0307Rf_20141111_1657.JPG


In [37]:



def parse_filename(filename):
    """
    Parse the filename to extract date and time.
    Handles filenames with optional '_web' before the extension.
    Returns a datetime object if the filename matches the pattern, otherwise None.
    """
    pattern = r'RC\d{4}[RL][a-z]?_(\d{8})_(\d{4})(_web)?.jpg'
    match = re.match(pattern, filename)
    if match:
        date_str, time_str, _ = match.groups()
        return datetime.strptime(date_str + time_str, '%Y%m%d%H%M')
    return None

def get_file_size(file_path):
    """
    Get and print the file size of the given image file.
    """
    size_bytes = os.path.getsize(file_path)
    size_kb = size_bytes / 1024
    print(f"File: {file_path}, Size: {size_kb:.2f} KB")
    return size_kb

def resize_image(image, scale_percent):
    """
    Resize the given image by the specified scale percentage.
    """
    width = int(image.shape[1] * scale_percent / 100)
    height = int(image.shape[0] * scale_percent / 100)
    dim = (width, height)
    return cv2.resize(image, dim, interpolation=cv2.INTER_AREA)
    
def filter_images(folder_path, start_time, end_time):
    """
    Filter images in the folder based on the specified time range.
    Returns a list of filenames that fall within the time range, sorted by datetime.
    """
    filtered_files = []
    for filename in os.listdir(folder_path):
        file_datetime = parse_filename(filename)
        if file_datetime and start_time <= file_datetime.time() <= end_time:
            filtered_files.append((file_datetime, os.path.join(folder_path, filename)))

    # Sort the list by datetime
    filtered_files.sort(key=lambda x: x[0])

    # Extract just the file paths
    sorted_files = [file[1] for file in filtered_files]
    print(f'The Directory {folder_path} contains {len(sorted_files)} within the specified start_time, end_time range')
    print(f'The first image is {sorted_files[0]} and the last image of the series is {sorted_files[-1]}')
    return sorted_files


def put_date_on_image(img, date_str):
    """
    Puts the date string on the bottom-left corner of the image.
    """
    font = cv2.FONT_HERSHEY_SIMPLEX
    font_scale = 2
    font_color = (255, 255, 255)  # white color
    line_type = 3

    cv2.putText(img, date_str, 
                (10, img.shape[0] - 10),  # position at bottom left
                font, 
                font_scale, 
                font_color, 
                line_type)

# def create_timelapse(site_name, image_files, out_dir = os.getcwd() frame_rate=30):
#     """
#     Create a timelapse video from the given image files.
#     Includes the site name and creation date in the video filename.
#     """
#     print(f'creating timelapse beginning with {image_files[0]} and ending with {image_files[-1]}') 
#     # Get the current date for the filename
#     current_date = datetime.now().strftime('%Y%m%d')
#     output_file = f'{out_dir + os.sep}{site_name}_timelapse_{current_date}.avi'  # Changed to .avi
#     output_file = os.path.join(os.getcwd(), output_file)  # Ensuring path compatibility

#     # Read the first image to determine the video resolution
#     frame = cv2.imread(image_files[0])
#     height, width, layers = frame.shape
#     size = (width, height)

#     # Create VideoWriter object with XVID codec
#     out = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*'XVID'), frame_rate, size)
#     n_images = 1
#     nn = len(image_files)
#     for filename in image_files:
#         print(f'processing image {filename} - {n_images} out of {nn}') 
#         img = cv2.imread(filename)
#         date_time = parse_filename(os.path.basename(filename))
#         date_str = date_time.strftime('%Y-%m-%d %H:%M')
#         put_date_on_image(img, date_str)
#         out.write(img)
#         n_images = n_images+1

#     out.release()
#     return output_file

def create_timelapse(site_name, image_files,out_dir = os.getcwd(),frame_rate=7, resize_percent=100):
    """
    Create a timelapse video, with options to specify an out_dir,select frame rate, and resize images by percent
    default framerate = 7 frames per second or 1 week per second if images are sorted by time of day to utilize a single image
    per day, which is recommended to reduce changes in water elevation and flickering from variable lighting at different times of day
    
    """
    current_date = datetime.now().strftime('%Y%m%d')
    output_file = f'{site_name}_timelapse_{current_date}.mp4'
    output_file = os.path.join(out_dir, output_file)

    if len(image_files) == 0:
        print("No images to process.")
        return None
        # Initialize VideoWriter with the first frame's dimensions after resizing
    first_frame = cv2.imread(image_files[0])
    if resize_percent != 100:
        first_frame = resize_image(first_frame, resize_percent)
    height, width, layers = first_frame.shape
    size = (width, height)
    out = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*'mp4v'), frame_rate, size)
    #out = cv2.VideoWriter(output_file, cv2.VideoWriter_fourcc(*'XVID'), frame_rate, size) # if you want .avi format
    n_images = 1
    nn = len(image_files)
    for filename in image_files:
        print(f'processing image {filename} - {n_images} out of {nn}') 
        img = cv2.imread(filename)
        if resize_percent != 100:
            img = resize_image(img, resize_percent)
        date_time = parse_filename(os.path.basename(filename))
        date_str = date_time.strftime('%Y-%m-%d %H:%M')
        put_date_on_image(img, date_str)
        out.write(img)
        n_images = n_images+1
        #get_file_size(filename)

    out.release()
    vid_size = get_file_size(output_file)
    
    return output_file



In [38]:
# Example usage
folder_path = image_dir22_web  # Replace with your images directory path
site_name = 'RC0220Ra_web'  # Replace with your site name

'''
Due to variable lighting in the canyon throughout the day, and the variable water levels,
its advisable to limit the timelapse to a single image per day rather than 5 per day. utlizing
the start_time and end_time arguments. All times in MST military time 00:00 - 24:59 %H:%M

If not using web-res images, is suggested to re-size images by at least 25% to prevent very large video files
'''
start_time = datetime.strptime('10:00', '%H:%M').time()
end_time = datetime.strptime('13:00', '%H:%M').time()

resize_percent = 100  # Resize images to 50% of their original size
print('filtering images')
filtered_files = filter_images(folder_path, start_time, end_time)
print('Images filtered')
print('creating timelapse')
filtered_files = filtered_files[0:100] # to test script on subset of data, use this line, otherwise comment out for full timelapse
video_filename = create_timelapse(site_name, filtered_files,out_dir = os.getcwd(), frame_rate=7, resize_percent=resize_percent)
if video_filename:
    print(f"Timelapse video created: {video_filename}")

filtering images
The Directory M:/Ryan_Lima/Remote Cameras/web_res/RC0220Ra contains 2872 within the specified start_time, end_time range
The first image is M:/Ryan_Lima/Remote Cameras/web_res/RC0220Ra\RC0220Ra_19960216_1000_web.jpg and the last image of the series is M:/Ryan_Lima/Remote Cameras/web_res/RC0220Ra\RC0220Ra_20160514_1147_web.jpg
Images filtered
creating timelapse
processing image M:/Ryan_Lima/Remote Cameras/web_res/RC0220Ra\RC0220Ra_19960216_1000_web.jpg - 1 out of 100
processing image M:/Ryan_Lima/Remote Cameras/web_res/RC0220Ra\RC0220Ra_19960217_1000_web.jpg - 2 out of 100
processing image M:/Ryan_Lima/Remote Cameras/web_res/RC0220Ra\RC0220Ra_19960218_1000_web.jpg - 3 out of 100
processing image M:/Ryan_Lima/Remote Cameras/web_res/RC0220Ra\RC0220Ra_19960219_1000_web.jpg - 4 out of 100
processing image M:/Ryan_Lima/Remote Cameras/web_res/RC0220Ra\RC0220Ra_19960220_1000_web.jpg - 5 out of 100
processing image M:/Ryan_Lima/Remote Cameras/web_res/RC0220Ra\RC0220Ra_19960221_